# 卷积神经网络

In [1]:
# 卷积神经网络
# ConvNet with MNIST
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
def conv_relu(inputs, filters, ksize, strides, padding, scope_name):
    """计算conv+relu的结果"""
    with tf.variable_scope(scope_name, reuse=tf.AUTO_REUSE) as scope:
        in_channels = inputs.shape[-1]
        kernel = tf.get_variable(name='kernel', 
                                shape=[ksize, ksize, in_channels, filters],
                                initializer=tf.truncated_normal_initializer())
        biases = tf.get_variable(name='biases',
                                shape=[filters],
                                initializer=tf.random_normal_initializer())
        conv = tf.nn.conv2d(inputs, kernel=kernel, strides=[1, strides, strides, 1], padding=padding)
    return tf.nn.relu(conv + biases, name=scope.name)

In [3]:
def maxpool(inputs, ksize, stride, padding='VALID', scope_name='pool'):
    '''A method that does max pooling on inputs'''
    with tf.variable_scope(scope_name, reuse=tf.AUTO_REUSE) as scope:
        pool = tf.nn.max_pool(inputs, 
                            ksize=[1, ksize, ksize, 1], 
                            strides=[1, stride, stride, 1],
                            padding=padding)
    return pool

In [4]:
def fully_connected(inputs, out_dim, scope_name='fc'):
    with tf.variable_scope(scope_name, reuse=tf.AUTO_REUSE) as scope:
        in_dim = inputs.shape[-1]
        w = tf.get_variable(name='weights', shape=[in_dim, out_dim], initializer=tf.truncated_normal_initializer())
        b = tf.get_variable(name='biases', shape=[out_dim], initializer=tf.constant_initializer(0.0))
        out = tf.matmul(inputs, w) + b
    return out

In [ ]:
class ConvNet(object):
    def __init__(self):
        self.lr = 1e-3
        self.batch_size = 128
        self.keep_prob = tf.constant(0.75)
        self.gstep = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')
        
        self.n_classes = 10
        self.skip_step = 20
        self.n_test = 10000
        self.training = True
        
        self.path = './MNIST/'
        
    def load_data(self):